## Wolffian duct differentiation - epithelium

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import seaborn as sns
import scipy.stats
import anndata
import os

import scipy as scipy
import scipy as sp
import pickle as pkl
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from statsmodels.nonparametric.smoothers_lowess import lowess
from numpy import asarray as ar
from collections import Counter
import networkx as nx
import igraph
import glob

In [ ]:
print(anndata.__version__)

In [ ]:
males_post10pcw = sc.read('/nfs/team292/vl6/FetalReproductiveTract/males_post10pcw.20240326.h5ad')
males_post10pcw.shape

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(6,6), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
sc.pl.umap(males_post10pcw, color="celltype")

In [ ]:
sc.pl.umap(males_post10pcw, color="celltype", 
           groups = ['CaputEpididymis_Epithelium', 'Corpus/CaudaEpididymis_Epithelium',
                                                                         ])

In [ ]:
males_post10pcw = males_post10pcw[[i in ['CaputEpididymis_Epithelium', 'Corpus/CaudaEpididymis_Epithelium']
                                    for i in males_post10pcw.obs['celltype']]]

In [ ]:
sc.pl.umap(males_post10pcw, color="celltype")

In [ ]:
import anndata
adata = anndata.AnnData(X = males_post10pcw.raw.X, obs = males_post10pcw.obs, var = males_post10pcw.raw.var)
adata

In [ ]:
adata = adata[[i == 'G1' for i in adata.obs['phase']]]

In [ ]:
adata.obs['stage_pcw'].value_counts(dropna = False)

In [ ]:
# Random downsampling per cell type 
import random
from itertools import chain
def downsample(adata, labels, n): 
    
    myindex = adata.obs[labels].value_counts().index 
    myvalues = adata.obs[labels].value_counts().values
    clusters = pd.Series(myvalues, index = myindex)
    
    # Find clusters with > n cells 
    cl2downsample = clusters.index[ clusters.values > n ]

    # save all barcode ids from small clusters
    holder = []
    holder.append( adata.obs_names[[ i not in cl2downsample for i in adata.obs[labels] ]] ) 

    # randomly sample n cells in the cl2downsample
    for cl in cl2downsample:
        print(cl)
        cl_sample = adata[[ i == cl for i in adata.obs[labels]]].obs_names
        
        cl_downsample = random.sample(set(cl_sample), n )
        holder.append(cl_downsample)
    
    # samples to include
    samples = list(chain(*holder))

    # Filter adata_count
    adata = adata[[ i in samples for i in adata.obs_names ]]
    return adata

In [ ]:
## import reproductive tract utils functions
import sys
cwd = '/nfs/team292/vl6/Experiments/Utils/'
sys.path.append(cwd)

import reptract_utils
import reptract_genes

In [ ]:
adata.shape

In [ ]:
adata.raw = adata.copy()

In [ ]:
sc.pp.filter_genes(adata, min_counts=10)

In [ ]:
#adata = reptract_utils.per_gene_analysis(adata)

In [ ]:
adata = reptract_utils.normalize_log_transform(adata)

In [ ]:
adata = reptract_utils.hvgs_pca_umap(adata)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'sample', 
                         'n_genes', 'percent_mito', 'percent_ribo', 'donor',  'doublet_scores',
                        'celltype', 'phase'], ncols = 2, wspace = 0.3, 
          color_map = 'OrRd')

In [ ]:
sc.pl.umap(adata, color = ['PAX8', 'GATA3', 'MUC6', 'ESR1', 'VIL1', 'HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'RXFP2', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2',  
                          'DLK1',], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 3)

In [ ]:
sc.tl.leiden(adata, resolution = 5)
sc.pl.umap(adata, color = ['leiden', 'doublet_scores', 'n_genes'], legend_loc = 'on data', color_map = 'OrRd')

In [ ]:
sc.pl.umap(adata, color = 'leiden', 
          groups = ['50', '20', '2', '23', '24', '28', '54', '36', '40', '52', '43'])

In [ ]:
adata = adata[[i not in ['50', '20', '2', '23', '24', '28', '54', '36', '40', '52', '43'] for i in adata.obs['leiden']]]

In [ ]:
adata.obs['stage_pcw'].value_counts()

In [ ]:
# adata = downsample(adata, 'stage_pcw', 1000)

In [ ]:
adata = anndata.AnnData(X = adata.raw.X, var = adata.raw.var, obs = adata.obs)

In [ ]:
adata.raw = adata.copy()
sc.pp.filter_genes(adata, min_counts=10)
#adata = reptract_utils.per_gene_analysis(adata)
adata = reptract_utils.normalize_log_transform(adata)
adata = reptract_utils.hvgs_pca_umap(adata)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'sample', 
                         'n_genes', 'donor',  'doublet_scores',
                        'celltype'], ncols = 2, wspace = 0.3, 
          color_map = 'viridis')

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'donor'], ncols = 2, wspace = 0.3, 
          color_map = 'viridis')

In [ ]:
sc.pl.umap(adata, color = ['PAX8', 'GATA3', 'MUC6', 'ESR1', 'VIL1', 'HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2',  
                          'DLK1'
                          ], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 4)

In [ ]:
# adata = anndata.AnnData(X = adata.raw.X, var = adata.raw.var, obs = adata.obs)

In [ ]:
# adata.raw = adata.copy()
# sc.pp.filter_genes(adata, min_counts=5)
# #adata = reptract_utils.per_gene_analysis(adata)
# adata = reptract_utils.normalize_log_transform(adata)
# adata = reptract_utils.hvgs_pca_umap(adata)

In [ ]:
# sc.pl.umap(adata, color=['stage', 'donor'], ncols = 2, wspace = 0.3, 
#           color_map = 'viridis')

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
# Extract PCA matrix and batch array
n_pcs = 15 # Principal components used to perform dimensionality reduction 
pca = adata.obsm['X_pca'][:, 0:(n_pcs)]
batch = adata.obs['donor'] # Batch ID, in this case stage

In [ ]:
%%R -i pca -i batch -o hem

library(harmony)
library(magrittr)
set.seed(1000)
hem <- HarmonyMatrix(pca, batch, theta=0, lambda = 6, do_pca=FALSE, verbose = FALSE)  
hem = data.frame(hem)

In [ ]:
# Add harmony values to the anndata object --> overwriting PCA 
adata.obsm['X_pca'] = hem.values 

In [ ]:
sc.pp.neighbors(adata, n_pcs = n_pcs, random_state= 123)
sc.tl.umap(adata, min_dist=0.6, spread = 0.8)

In [ ]:
sc.pl.umap(adata, color = ['PAX8', 'GATA3', 'MUC6', 'ESR1', 'VIL1', 'HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2',  
                          'DLK1'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'sample', 
                         'n_genes', 'donor',  'doublet_scores', 'percent_mito',
                        'celltype', 'percent_mito'], ncols = 2, wspace = 0.3, 
          color_map = 'viridis')

In [ ]:
sc.tl.leiden(adata, resolution = 0.3)
sc.pl.umap(adata, color = 'leiden', legend_loc = 'on data')

In [ ]:
sc.tl.draw_graph(adata)

In [ ]:
sc.pl.draw_graph(adata, color='leiden', legend_loc='on data')


In [ ]:
sc.pl.draw_graph(adata, color='n_genes', color_map='OrRd')


In [ ]:
sc.tl.paga(adata, groups='leiden')


In [ ]:
sc.pl.paga(adata, color=['leiden', 'VIL1', 'GATA3', 'EMX2'])


In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')


In [ ]:
sc.pl.draw_graph(adata, color='leiden', legend_loc='on data')


In [ ]:
import anndata
bdata = anndata.AnnData(X = adata.raw.X, var = adata.raw.var, obs = adata.obs)

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
for col in bdata.obs.columns: 
    if col != 'leiden':
        del bdata.obs[col]

In [ ]:
%%R -i bdata
bdata

In [ ]:
%%R -o mrks

library(SoupX)
counts <- assay(bdata, "X")
colnames(counts) <- colnames(bdata)
rownames(counts) <- rownames(bdata)
mrks = quickMarkers(counts, colData(bdata)$leiden, N = 100) # where clusters is the cell type assignment and 100 means I want the top 100 genes per cluster that pass the hypergeometric test

In [ ]:
sc.pl.umap(adata, color = ['stage_pcw','leiden', 'donor'],  ncols = 2)

In [ ]:
cluster = mrks[mrks['cluster'] == '0']['gene'].to_list()[0:15]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
sc.pl.umap(adata, color = ['n_genes', 'n_counts', 'percent_mito'], color_map = 'OrRd')

In [ ]:
sc.pl.umap(adata, color = 'leiden', legend_loc = 'on data')

In [ ]:
annots = {'0' : 'EfferentDuctules_Epididymis', '1' : 'Epididymis/VasDeferenes_Epithelium', '2' : 'Epididymis/VasDeferenes_Epithelium', 
        '3' : 'EfferentDuctules_Epididymis', '4' : 'EfferentDuctules_Epididymis', 
        '5' : 'EfferentDuctules_Epididymis', '6' : 'Epididymis/VasDeferenes_Epithelium'}

In [ ]:
adata.obs['epi_wolffian_lowres'] = adata.obs.leiden.map(annots)

In [ ]:
sc.pl.umap(adata, color = ['stage_pcw', 'donor', 'epi_wolffian_lowres'], 
          ncols = 2, save = '_epi_wolffian')

In [ ]:
sc.pl.umap(adata, color = ['epi_wolffian_lowres'], 
         )

In [ ]:
adata.write('/nfs/team292/vl6/FetalReproductiveTract/wolffian_epi_late_post10pcw.h5ad')